In [1]:
import torch
import random
import copy
import itertools
import numpy as np
import utils

from sklearn.metrics import classification_report
from LIM_bert import LIMBERTClassifier
from ii_benchmark import IIBenchmarkMoNli

utils.fix_random_seeds()

In [2]:
benchmark = IIBenchmarkMoNli(
        variable_names=['LEX'],
        data_parameters={
            'train_size': 10000, 'test_size': 10000
        },
        model_parameters={
            'weights_name': 'ishan/bert-base-uncased-mnli',
            'max_length': 128,
            'n_classes': 2,
            'hidden_dim': 768,
            'target_dims':[0,786],
            'debug':False
        },
        training_parameters={
            'warm_start': False, 'max_iter': 3, 'batch_size': 32, 'n_iter_no_change': 10000, 
            'shuffle_train': True, 'eta': 0.00002
        },
        seed=60
)
LIM_bert = benchmark.create_model()

Some weights of the model checkpoint at ishan/bert-base-uncased-mnli were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: CUDA out of memory. Tried to allocate 9.00 GiB (GPU 0; 8.00 GiB total capacity; 0 bytes already allocated; 6.99 GiB free; 0 bytes reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
LIM_trainer = benchmark.create_classifier(LIM_bert)

In [ ]:
X_base_train, y_base_train, X_sources_train, y_IIT_train, interventions = benchmark.load_datasets()[0]

In [ ]:
_ = LIM_trainer.fit(
            X_base_train, 
            y_base_train
            )

In [ ]:
preds = LIM_trainer.predict(X_base_train)
print(classification_report(y_base_train, preds.cpu()))

In [ ]:
alignment  = {0:[{"layer":0, "start":0, "end":64}]}

In [ ]:
LIM_trainer.model.set_analysis_mode(True)
_ = LIM_trainer.fit(
    X_base_train, 
    y_base_train, 
    iit_data=(
                X_sources_train,
                y_IIT_train,
                interventions
            ),
    intervention_ids_to_coords=alignment)

In [ ]:
evaluation = benchmark.evaluate(blackbox_LIM, alignment)
benchmark.display_evaluations(evaluation)